<h2>Conflict and Disaster API Usage and Data Scraping</h2>
<h4>Using the <a href="https://apidocs.acleddata.com/get_started.html">ACLED API</a></h4>
<p>This data was originally accessed on March 1, 2024 and will be continually updated as part of a data pipeline in the backend of a model-mapping web application. Armed Conflict Location & Event Data Project (ACLED); www.acleddata.com</p>

<a href="https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2023/06/ACLED_Codebook_2023.pdf">Data Columns Codebook</a>

In [2]:
# General imports
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import requests
import json
from io import StringIO
from typing import Union

In [3]:
# Setting global API Variables
BASE_URL = "https://api.acleddata.com/"
ACLED_API_KEY = "VBNw-*4nvzHEC87xu2qQ"
USER_EMAIL = "nabji@berkeley.edu"
BASE_PARAMS = {
    "email": USER_EMAIL,
    "key": ACLED_API_KEY,
}

fields = [
    "event_id_cnty",
    "event_date",
    "year",
    "time_precision",
    "disorder_type",
    "event_type",
    "sub_event_type",
    "actor1",
    "assoc_actor_1",
    "inter1",
    "actor2",
    "assoc_actor_2",
    "inter2",
    "interaction",
    "civilian_targeting",
    "iso",
    "region",
    "country",
    "admin1",
    "admin2",
    "admin3",
    "location",
    "latitude",
    "longitude",
    "geo_precision",
    "source",
    "source_scale",
    "notes",
    "fatalities",
    "tags",
    "timestamp"
]

BASE_PARAMS["fields"] = "|".join(fields)
BASE_PARAMS

{'email': 'nabji@berkeley.edu',
 'key': 'VBNw-*4nvzHEC87xu2qQ',
 'fields': 'event_id_cnty|event_date|year|time_precision|disorder_type|event_type|sub_event_type|actor1|assoc_actor_1|inter1|actor2|assoc_actor_2|inter2|interaction|civilian_targeting|iso|region|country|admin1|admin2|admin3|location|latitude|longitude|geo_precision|source|source_scale|notes|fatalities|tags|timestamp'}

In [12]:
# Creating a function to request data from ACLED
def get_ACLED_data(country: str, year: int, page: int) -> Union[pd.DataFrame, bool]:
    """A function which, given a country and year will return all fields of ACLED satisfying those parameters
    
    Parameters
    ----------
    country: str
        The country from which we are requesting data
    year: int
        The year from which we are requesting data. year <= 2024
    page: int
        The page number to paginate API calls
    
    Return Value
    ------------
    type: pd.DataFrame | bool
        A Pandas DataFrame containing information for all ACLED columns satisying the given parameters. If the request fails, return False
    """
    # Modify the base parameters
    new_params = BASE_PARAMS.copy()
    new_params["country"] = country
    new_params["year"] = year
    print(country)
    
    # Request the data as a CSV file and pass our paramenters as an argument (params=)
    response = requests.get(f"https://api.acleddata.com/acled/read.csv?limit=5000&page={page}", params=new_params)
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        return_df = pd.read_csv(csv_data)
        return return_df
    else:
        return False

In [ ]:
get_ACLED_data(country="Georgia", year=2023)

In [11]:
countries_str = """China 1/2018-Present
Japan 1/2018-Present
Mongolia 1/2018-Present
North Korea 1/2018-Present
South Korea 1/2018-Present
Taiwan 1/2018-Present
Afghanistan 1/2017-Present
Armenia 1/2018-Present
Azerbaijan 1/2018-Present
Georgia 1/2018-Present
Kazakhstan 1/2018-Present
Kyrgyzstan 1/2018-Present
Tajikistan 1/2018-Present
Turkmenistan 1/2018-Present
Uzbekistan 1/2018-Present
Albania 1/2018-Present
Andorra 1/2020-Present
Austria 1/2020-Present
Bailiwick of Guernsey 1/2020-Present
Bailiwick of Jersey 1/2020-Present
Belarus 1/2018-Present
Belgium 1/2020-Present
Bosnia and Herzegovina 1/2018-Present
Bulgaria 1/2018-Present
Croatia 1/2018-Present
Cyprus 1/2018-Present
Czech Republic 1/2020-Present
Denmark 1/2020-Present
Estonia 1/2020-Present
Faroe Islands 1/2020-Present
Finland 1/2020-Present
France 1/2020-Present
Germany 1/2020-Present
Gibraltar 1/2020-Present
Greece 1/2018-Present
Greenland 1/2020-Present
Hungary 1/2020-Present
Iceland 1/2020-Present
Ireland 1/2020-Present
Isle of Man 1/2020-Present
Italy 1/2020-Present
Kosovo 1/2018-Present
Latvia 1/2020-Present
Liechtenstein 1/2020-Present
Lithuania 1/2020-Present
Luxembourg 1/2020-Present
Malta 1/2020-Present
Moldova 1/2018-Present
Monaco 1/2020-Present
Montenegro 1/2018-Present
Netherlands 1/2020-Present
North Macedonia 1/2018-Present
Norway 1/2020-Present
Poland 1/2020-Present
Portugal 1/2020-Present
Romania 1/2018-Present
Russia 1/2018-Present
San Marino 1/2020-Present
Serbia 1/2018-Present
Slovakia 1/2020-Present
Slovenia 1/2020-Present
Spain 1/2020-Present
Sweden 1/2020-Present
Switzerland 1/2020-Present
Ukraine 1/2018-Present
United Kingdom 1/2020-Present
Vatican City 1/2020-Present
Syria 1/2017-Present
Iraq 1/2016-Present
Turkey 1/2016-Present
Iran 1/2016-Present
Saudi Arabia 1/2015-Present
Yemen 1/2015-Present
United Arab Emirates 1/2016-Present
Israel 1/2016-Present
Jordan 1/2016-Present
Palestine 1/2016-Present
Lebanon 1/2016-Present
Oman 1/2016-Present
Kuwait 1/2016-Present
Qatar 1/2016-Present
Bahrain 1/2016-Present
South & Southeast Asia
Bangladesh 1/2010-Present
Bhutan 1/2020-Present
Brunei 1/2020-Present
Cambodia 1/2010-Present
East Timor 1/2020-Present
India 1/2016-Present
Indonesia 1/2015-Present
Laos 1/2010-Present
Malaysia 1/2018-Present
Maldives 1/2020-Present
Myanmar 1/2010-Present
Nepal 1/2010-Present
Pakistan 1/2010-Present
Philippines 1/2016-Present
Singapore 1/2020-Present
Sri Lanka 1/2010-Present
Thailand 1/2010-Present
Vietnam 1/2010-Present
Algeria 1/1997-Present
Angola 1/1997-Present
Benin 1/1997-Present
Botswana 1/1997-Present
British Indian Ocean Territory 1/2018-Present
Burkina Faso 1/1997-Present
Burundi 1/1997-Present
Cameroon 1/1997-Present
Cape Verde 1/2020-Present
Central African Republic 1/1997-Present
Chad 1/1997-Present
Comoros 1/2020-Present
Democratic Republic of Congo 1/1997-Present
Republic of Congo 1/1997-Present
Cote d'Ivoire 1/1997-Present
Djibouti 1/1997-Present
Egypt 1/1997-Present
Equatorial Guinea 1/1997-Present
Eritrea 1/1997-Present
eSwatini 1/1997-Present
Ethiopia 1/1997-Present
Gabon 1/1997-Present
Gambia 1/1997-Present
Ghana 1/1997-Present
Guinea 1/1997-Present
Guinea-Bissau 1/1997-Present
Kenya 1/1997-Present
Lesotho 1/1997-Present
Liberia 1/1997-Present
Libya 1/1997-Present
Madagascar 1/1997-Present
Malawi 1/1997-Present
Mali 1/1997-Present
Mauritania 1/1997-Present
Mauritius 1/2020-Present
Mayotte 1/2020-Present
Morocco 1/1997-Present
Mozambique 1/1997-Present
Namibia 1/1997-Present
Niger 1/1997-Present
Nigeria 1/1997-Present
Réunion 1/2020-Present
Rwanda 1/1997-Present
Saint Helena, Ascension, and Tristan da Cunha 1/2020-Present
São Tomé and Príncipe 1/2020-Present
Senegal 1/1997-Present
Seychelles 1/2020-Present
Sierra Leone 1/1997-Present
Somalia 1/1997-Present
South Africa 1/1997-Present
South Sudan 1/2011-Present
Sudan 1/1997-Present
Tanzania 1/1997-Present
Togo 1/1997-Present
Tunisia 1/1997-Present
Uganda 1/1997-Present
Zambia 1/1997-Present
Zimbabwe 1/1997-Present
Latin America & the Caribbean
Puerto Rico (USA) 1/2018-Present
Guadeloupe (France) 1/2018-Present
Martinique (France) 1/2018-Present
French Guiana (France) 1/2018-Present
Curaçao (Netherlands) 1/2018-Present
Aruba (Netherlands) 1/2018-Present
US Virgin Islands (USA) 1/2018-Present
Cayman Islands 1/2018-Present
Sint Maarten (Netherlands) 1/2018-Present
Turks and Caicos (UK) 1/2018-Present
British Virgin Islands (UK) 1/2018-Present
Caribbean Netherlands (Bonaire, Sint Eustatius, and Saba)
Anguilla (UK) 1/2018-Present
Montserrat (UK) 1/2018-Present
Falkland Islands (UK) 1/2018-Present
Brazil 1/2018-Present
Mexico 1/2018-Present
Colombia 1/2018-Present
Argentina 1/2018-Present
Peru 1/2018-Present
Venezuela 1/2018-Present
Chile 1/2018-Present
Guatemala 1/2018-Present
Ecuador 1/2018-Present
Cuba 1/2018-Present
Bolivia 1/2018-Present
Haiti 1/2018-Present
Dominican Republic 1/2018-Present
Honduras 1/2018-Present
Paraguay 1/2018-Present
El Salvador 1/2018-Present
Nicaragua 1/2018-Present
Costa Rica 1/2018-Present
Panama 1/2018-Present
Uruguay 1/2018-Present
Jamaica 1/2018-Present
Trinidad and Tobago 1/2018-Present
Guyana 1/2018-Present
Suriname 1/2018-Present
Bahamas 1/2018-Present
Belize 1/2018-Present
Barbados 1/2018-Present
Saint Lucia 1/2018-Present
St. Vincent & Grenadines 1/2018-Present
Grenada 1/2018-Present
Antigua & Barbuda 1/2018-Present
Dominica 1/2018-Present
Saint-Barthelemy 1/2018-Present
Saint-Martin 1/2018-Present
Saint Kitts & Nevis 1/2018-Present
South Georgia and the South Sandwich Islands (UK) 1/2018-Present
Canada 1/2021-Present
Bermuda (UK) 1/2021-Present
Saint Pierre and Miquelon (France) 1/2021-Present
United States 1/2020-Present
Australia 1/2021-Present
New Zealand 1/2021-Present
Fiji 1/2021-Present
New Caledonia (France) 1/2021-Present
Vanuatu 1/2021-Present
Solomon Islands 1/2021-Present
Papua New Guinea 1/2021-Present
Micronesia 1/2021-Present
Guam (US) 1/2021-Present
Nauru 1/2021-Present
Marshall Islands 1/2021-Present
Kiribati 1/2021-Present
Palau 1/2021-Present
Northern Mariana Islands (US) 1/2021-Present
American Samoa (US) 1/2021-Present
Cook Islands 1/2021-Present
French Polynesia (France) 1/2021-Present
Norfolk Island (Australia) 1/2021-Present
Niue 1/2021-Present
Samoa 1/2021-Present
Tonga 1/2021-Present
Tokelau (New Zealand) 1/2021-Present
Tuvalu 1/2021-Present
Pitcairn (UK) 1/2021-Present
Wallis and Futuna (France) 1/2021-Present
Heard Island and McDonald Islands (Australia) 1/2021-Present
Cocos (Keeling) Islands (Australia) 1/2021-Present
Christmas Island (Australia) 1/2021-Present
US Outlying Minor Islands 1/2021-Present
Antarctica 1/2021-Present"""

countries_lst = [country.split(" 1")[0] for country in countries_str.split("\n")]
countries_lst = ["%20".join(country.split(" ")) for country in countries_lst]
countries_lst

['China',
 'Japan',
 'Mongolia',
 'North%20Korea',
 'South%20Korea',
 'Taiwan',
 'Afghanistan',
 'Armenia',
 'Azerbaijan',
 'Georgia',
 'Kazakhstan',
 'Kyrgyzstan',
 'Tajikistan',
 'Turkmenistan',
 'Uzbekistan',
 'Albania',
 'Andorra',
 'Austria',
 'Bailiwick%20of%20Guernsey',
 'Bailiwick%20of%20Jersey',
 'Belarus',
 'Belgium',
 'Bosnia%20and%20Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech%20Republic',
 'Denmark',
 'Estonia',
 'Faroe%20Islands',
 'Finland',
 'France',
 'Germany',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Hungary',
 'Iceland',
 'Ireland',
 'Isle%20of%20Man',
 'Italy',
 'Kosovo',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Moldova',
 'Monaco',
 'Montenegro',
 'Netherlands',
 'North%20Macedonia',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'San%20Marino',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Ukraine',
 'United%20Kingdom',
 'Vatican%20City',
 'Syria',
 'Iraq',
 'Turkey',
 'Iran',
 'S

In [14]:
counter = 2
df = get_ACLED_data(country="|".join(countries_lst), year=2023, page=1)
while True:
    new_df = get_ACLED_data(country="|".join(countries_lst), year=2023, page=counter)
    df = pd.concat([df, new_df], ignore_index=True)
    if len(new_df) < 5000:
        break
    counter += 1

China|Japan|Mongolia|North%20Korea|South%20Korea|Taiwan|Afghanistan|Armenia|Azerbaijan|Georgia|Kazakhstan|Kyrgyzstan|Tajikistan|Turkmenistan|Uzbekistan|Albania|Andorra|Austria|Bailiwick%20of%20Guernsey|Bailiwick%20of%20Jersey|Belarus|Belgium|Bosnia%20and%20Herzegovina|Bulgaria|Croatia|Cyprus|Czech%20Republic|Denmark|Estonia|Faroe%20Islands|Finland|France|Germany|Gibraltar|Greece|Greenland|Hungary|Iceland|Ireland|Isle%20of%20Man|Italy|Kosovo|Latvia|Liechtenstein|Lithuania|Luxembourg|Malta|Moldova|Monaco|Montenegro|Netherlands|North%20Macedonia|Norway|Poland|Portugal|Romania|Russia|San%20Marino|Serbia|Slovakia|Slovenia|Spain|Sweden|Switzerland|Ukraine|United%20Kingdom|Vatican%20City|Syria|Iraq|Turkey|Iran|Saudi%20Arabia|Yemen|United%20Arab%20Emirates|Israel|Jordan|Palestine|Lebanon|Oman|Kuwait|Qatar|Bahrain|South%20&%20Southeast%20Asia|Bangladesh|Bhutan|Brunei|Cambodia|East%20Timor|India|Indonesia|Laos|Malaysia|Maldives|Myanmar|Nepal|Pakistan|Philippines|Singapore|Sri%20Lanka|Thailand|Vi

In [7]:
dataframes

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,IRN25775,2023-12-31,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Iran),Labor Group (Iran),6,...,Ahvaz,31.3190,48.6842,1,Free Union of Iranian Workers; Human Rights Ac...,Local partner-Other,"On 31 December 2023, retirees of the Social Se...",0,crowd size=no report,1704756262
1,IRN25776,2023-12-31,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Iran),Labor Group (Iran),6,...,Ahvaz,31.3190,48.6842,1,Human Rights Activists in Iran (HRA),Local partner-Other,"On 31 December 2023, retirees of Isfahan steel...",0,crowd size=no report,1704756262
2,IRN25777,2023-12-31,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Iran),Labor Group (Iran),6,...,Arak,34.0917,49.6892,1,Human Rights Activists in Iran (HRA),Local partner-Other,"On 31 December 2023, retirees of the Social Se...",0,crowd size=no report,1704756262
3,IRN25778,2023-12-31,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Iran),Labor Group (Iran),6,...,Bandar e Mahshahr,30.5566,49.1885,1,Eteraze Bazar; Human Rights Activists in Iran ...,Local partner-Other,"On 31 December 2023, workers of Bandar Mahshah...",0,crowd size=no report,1704756262
4,IRN25779,2023-12-31,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Iran),Labor Group (Iran),6,...,Haft Tappeh,32.0837,48.4100,1,Iranian Labour News Agency,National,"On 31 December 2023, workers of Pars Steel Com...",0,crowd size=no report,1704756262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,USA64207,2023-12-25,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (United States),Palestinian Group (United States),6,...,Sacramento,38.5799,-121.4962,1,CBS News,National,"On 25 December 2023, pro-Palestinian demonstra...",0,crowd size=no report,1704833918
4996,IDN14603,2023-12-25,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Indonesia),Labor Group (Indonesia),6,...,Bandung,-6.9222,107.6070,1,Detik,National,"On 25 December 2023, a group of street vendors...",0,crowd size=no report,1704833919
4997,PHL16359,2023-12-25,2023,1,Political violence,Battles,Armed clash,NPA: New People's Army,NaN,2,...,Malaybalay,8.1575,125.1277,1,ABS-CBN; CNN; GMA News; Manila Bulletin; Minda...,Subnational-National,"On 25 December 2023, 10 suspected NPA rebels, ...",10,NaN,1704833919
4998,PNG749,2023-12-25,2023,1,Political violence,Riots,Mob violence,Rioters (Papua New Guinea),Kerema Communal Group (Papua New Guinea),5,...,Port Moresby,-9.4440,147.1813,1,PNG Haus Bung,New media,"On 25 December 2023, Kerema and Engan groups c...",0,crowd size=no report,1704833919


In [18]:
df.loc[1:len(df) // 2].to_csv("csv/ACLED_DB_HALF_1")
df.loc[len(df) //2:].to_csv("csv/ACLED_DB_HALF_2")